# WHATEVER

#  🤣MOROKEI😋 - 2023

설명 (Description)

GYMX1 부터 GYMX9 까지의 업그레이드를 통해 마지막에 BETAZERO V0 가 된 겁니다.
CHAT GPT 4 의 도움을 굉장히 많이 받았습니다.

자세한 사항은 www.morokei.com/DeepLearning/BetaZero 에서 확인하세요.

20161728, 화공생명공학과, 서강대학교

Email : thenoblesse5322@gmail.com

2023.04.08

추가된 부분
1. 구조 대폭 수정 : legal moves만 가지고 움직이는 policy

신경망과 DQN 에이전트를 초기화합니다. 두 명의 에이전트가 경쟁하도록 설정하여 서로 학습하게 합니다.

게임 환경을 초기화하고 여러 에피소드를 통해 에이전트를 훈련시킵니다. 에이전트는 현재 상태에서 가능한 최적의 행동을 선택하고, 행동에 대한 결과로 다음 상태와 보상을 얻습니다. 이 경험들은 ReplayBuffer에 저장됩니다.

버퍼에 충분한 경험이 쌓이면, 에이전트는 이러한 경험들을 샘플링하여 신경망을 업데이트하고 학습합니다. 학습 과정에서 기대되는 Q-값과 목표 Q-값 사이의 차이를 최소화하도록 신경망의 가중치를 조정합니다.

목표 네트워크는 일정한 간격으로 로컬 네트워크의 가중치를 부드럽게 가져와 업데이트합니다. 이렇게 하여 학습 과정의 안정성을 유지하고, 최신 가중치를 목표 네트워크에 반영합니다.

각 에피소드가 종료될 때마다, 에이전트간의 스코어를 기록하고, 일정한 에피소드마다 평균 스코어를 출력하여 에이전트의 학습 진행 상황을 확인할 수 있습니다.

일정한 에피소드마다 신경망 모델과 ReplayBuffer를 저장합니다. 이를 통해 훈련 중간에 중단되더라도 이후에 훈련을 이어갈 수 있으며, 최적화된 모델을 사용하여 게임을 진행할 수 있습니다.

사용자와 상호작용하는 '실행' 함수를 통해, 모델을 학습시키거나, 하이퍼파라미터를 최적화하거나, 모델을 저장 및 불러오는 작업을 수행할 수 있습니다. 이를 통해 사용자는 에이전트의 학습 상태를 쉽게 관리하고, 원하는 시점에서 모델을 사용하여 게임을 진행할 수 있습니다.

# 1. CHESS PART

## Import Module Data

### Import Gym Module Data

In [1]:
import gym , random, time, os, sys, glob
import numpy as np, pandas as pd
import chess, datetime, chess.pgn
import pickle, re

경로 = os.getcwd() ; print ("경로 : ", 경로, "\n")
sys.path.insert(0, ""+경로+"/data/used modules/")

import gym_chess_morokei as gym_chess
from gym_chess_morokei.envs import Chess

print ("일반모듈경로 : " , os.path.dirname(gym.__file__), "\n")
print ("특별모듈경로 : ", os.path.dirname(gym_chess.__file__), "\n")

경로 :  D:\chess_ai\BetaZero (new)\BETAZERO V2+ 

일반모듈경로 :  C:\Users\ojs53\AppData\Local\Programs\Python\Python310\lib\site-packages\gym 

특별모듈경로 :  D:\chess_ai\BetaZero (new)\BETAZERO V2+/data/used modules\gym_chess_morokei 



### Import Torch Module Data

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

### Import Other Module Data

In [3]:
import math ; import matplotlib, matplotlib.pyplot as plt

from collections import namedtuple, deque
from itertools import count ; from PIL import Image

## Import Gym Environment Data

### 체스보드 생성 (LEGACY)

In [4]:
env = gym_chess._make_env()
observation = env.reset()
print(observation)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [5]:
print(type(env._board))
print(type(observation))

<class 'chess.Board'>
<class 'chess.Board'>


### 인코더 함수 정의

기존 인코더에, 텐서 하나를 새로 맨 앞에 추가 했다ㅏ. 맨 앞 텐서는 흑과 백 정보를 가지고 있다.

이게 없으면, 모델이 아무리 초고수더라도 자기가 백인지 흑인지 맨 처음에 찍어야 하는 불상사가 생기기 때문이다.

In [6]:
def encoder(chessboard_observation):
    tensor_observation = torch.zeros((14, 8, 8))

    for i in range(8):
        for j in range(8):
            piece = chessboard_observation.piece_at(i * 8 + j)
            if piece is not None:
                # Encode piece type and color as one-hot vector
                index = piece.piece_type - 1 + (piece.color - 1) * 6 + 1
                tensor_observation[index][i][j] = 1
            else:
                # Encode empty square as one-hot vector
                tensor_observation[7][i][j] = 1

    # Encode the current player's color
    if chessboard_observation.turn:  # White
        tensor_observation[0] = 1
    else:  # Black
        tensor_observation[0] = 0

    tensor_observation = tensor_observation.unsqueeze(0)  # add batch dimension
    return tensor_observation


In [7]:
print(encoder(observation))

tensor([[[[1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1.]],

         [[0., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 1., 1., 1.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.]],

         [[0., 1., 0., 0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0

In [8]:
'''

Chessboard UCI Table

8        [[56, 57, 58, 59, 60, 61, 62, 63],
7         [48, 49, 50, 51, 52, 53, 54, 55],
6         [40, 41, 42, 43, 44, 45, 46, 47],
5         [32, 33, 34, 35, 36, 37, 38, 39],
4         [24, 25, 26, 27, 28, 29, 30, 31],
3         [16, 17, 18, 19, 20, 21, 22, 23],
2         [8,  9,  10, 11, 12, 13, 14, 15],
1         [0,  1,  2,  3,  4,  5,  6,  7]]
           a   b   c   d   e   f   g   h
           
           
'''

'\n\nChessboard UCI Table\n\n8        [[56, 57, 58, 59, 60, 61, 62, 63],\n7         [48, 49, 50, 51, 52, 53, 54, 55],\n6         [40, 41, 42, 43, 44, 45, 46, 47],\n5         [32, 33, 34, 35, 36, 37, 38, 39],\n4         [24, 25, 26, 27, 28, 29, 30, 31],\n3         [16, 17, 18, 19, 20, 21, 22, 23],\n2         [8,  9,  10, 11, 12, 13, 14, 15],\n1         [0,  1,  2,  3,  4,  5,  6,  7]]\n           a   b   c   d   e   f   g   h\n           \n           \n'

### 아웃풋 / 리버스 아웃풋

#### 아웃풋 리스트 저장

In [9]:
# csv 파일 있으면 그거 쓰고 없으면 새로 리스트 만들기

csv_path = "data/모든 체스 UCI 목록.csv"

if os.path.exists(csv_path):
    outcome_df = pd.read_csv(csv_path)
    outcome = [chess.Move.from_uci(move) for move in outcome_df['moves']]
else:
    outcome = [chess.Move(0, 0)]

    for start in range(64):
        for end in range(64):
            if start == end:
                continue
            outcome.append(chess.Move(start, end))

    promotion_pairs = [(48, 56), (48, 57), (49, 56), (49, 57), (49, 58),
             (50, 57), (50, 58), (50, 59), (51, 58), (51, 59),
             (51, 60), (52, 59), (52, 60), (52, 61), (53, 60),
             (53, 61), (53, 62), (54, 61), (54, 62), (54, 63),
             (55, 62), (55, 63), (8, 0), (8, 1), (9, 0), (9, 1),
             (9, 2), (10, 1), (10, 2), (10, 3), (11, 2), (11, 3),
             (11, 4), (12, 3), (12, 4), (12, 5), (13, 4), (13, 5),
             (13, 6), (14, 5), (14, 6), (14, 7), (15, 6), (15, 7)]

    for pair in promotion_pairs:
        for promote_to in [chess.QUEEN, chess.ROOK, chess.BISHOP, chess.KNIGHT]:
            outcome.append(chess.Move(pair[0], pair[1], promotion=promote_to))

    # Save the outcome list to a CSV file
    (pd.DataFrame(outcome, columns=['moves'])).to_csv(csv_path, index=False)
print(len(outcome))

4209


총 가능 UCI 는 4208개로 측정되었고, 기권을 의미하는 0 --> 0 를 하나 추가하여 총 4209 개의 outcome 이 있다.

프로모션 되는 무브 저장
48 56)
48 57)
49 56)
49 57)
49 58)
50 57)
50 58)
50 59)
51 58)
51 59)
51 60)
52 59)
52 60)
52 61)
53 60)
53 61)
53 62)
54 61)
54 62)
54 63)
55 62)
55 63)
8 0)
8 1)
9 0)
9 1)
9 2)
10 1)
10 2 )
10 3)
11 2)
11 3)
11 4 )
12 3)
12 4 )
12 5)
13 4 )
13 5)
13 6)
14 5)
14 6)
14 7)
15 6)
15 7)

#### 리버스 아웃풋 함수 정의

In [10]:
def index_to_move(move_index):
    return outcome[move_index]

In [11]:
def legal_moves_indices(board):
    legal_moves = list(board.legal_moves)
    legal_moves_indices = [outcome.index(move) for move in legal_moves]
    return legal_moves_indices

In [12]:
legal_moves_indices(observation)

[401,
 399,
 81,
 79,
 968,
 904,
 840,
 776,
 712,
 648,
 584,
 520,
 976,
 912,
 848,
 784,
 720,
 656,
 592,
 528]

Computer_1 = []
Computer_2 = []
observation, reward, done, _ = env.step(action_move = chess.Move(outcome[Computer_1_선택결과]))
observation, reward, done, _ = env.step(action_move = chess.Move(outcome[Computer_2_선택결과]))

참고 :  INDEX NUMBERING

    0 : p
    1 : n
    2 : b
    3 : r
    4 : q
    5 : k
    6 : 빈공간
    7 : P
    8 : N
    9 : B
    10 : R
    11 : Q
    12 : K

In [13]:
print(Chess._rewards)
print("\n")
print(env._reward())

{'*': 0.0, '1/2-1/2': -0.5, '1-0': 1.0, '0-1': -1.0, 'Illegal Move': -1.0}


0.0


# 2. Torch Part

### Initialize CUDA

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
print("니 파이토치 버전 이랑 쿠다 호환 버전이다  : " , torch.          __version__, end="\n")
print("CUDA 할당 된 그래픽 카드다   : ", torch.cuda.get_device_name())

니 파이토치 버전 이랑 쿠다 호환 버전이다  :  1.13.1+cu117
CUDA 할당 된 그래픽 카드다   :  NVIDIA GeForce RTX 3070


## CNN 정의

0410
인코더에서 맨 첫번 째 텐서에 흑과 백의 정보를 추가 시켰다.
그래서 CNN 에서도 14개의 텐서를 받게 되고, 첫번 째 꺼는 그냥 8x8 로 계산시키고
나머지 13개는 원래 하던 대로 하는 코드로 변경했다.

0413
결과가 안좋아서
1. Residual connection 사용
2. hidden layer 추가 (아니 이전에 히든 레이어가 아예 없더라 ㅋㅋㅋㅋㅋㅋㅋㅋ 무슨 벌레색휘도 아니고 ㅋㅋㅋㅋㅋㅋㅋㅋ 🤣🤣🤣 )

3. 

### CNN

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.convs = nn.ModuleList([
            nn.Conv2d(14, 14, kernel_size=k, padding=0) for k in range(1, 9)
        ])

        self.hidden1 = nn.Sequential(
            nn.Conv2d(14, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64)
        )
        self.hidden2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64)
        )
        self.hidden3 = nn.Linear(64 * 8 * 8, 32 * 8 * 8)

        self.fc1 = nn.Linear(32 * 8 * 8, 2048)
        self.fc2 = nn.Linear(2048, 4209)

    def forward(self, x):
        conv_outputs = []
        max_height, max_width = 8, 8

        for conv in self.convs:
            out = torch.relu(conv(x))
            height_diff = max_height - out.size(2)
            width_diff = max_width - out.size(3)
            out = F.pad(out, (0, width_diff, 0, height_diff))
            conv_outputs.append(out)

        x = torch.cat(conv_outputs, dim=1)
        x = self.hidden1(x) + x
        x = F.relu(x)
        x = self.hidden2(x) + x
        
        x = x.view(-1, 64 * 8 * 8)
        x = self.hidden3(x)
        x = F.relu(x)
        x = x.view(-1, 32, 8, 8)

        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## DQN 

1. self.trained = False 를 통해서 빈 모델을 저장하게 되는 일을 막음

### DQN 정의

In [17]:
class DQN:
    def __init__(self, cnn, buffer_size=1000000, batch_size=10000, gamma=0.99, learning_rate=0.001, tau=0.01, device=device):
        self.local_net = cnn.to(device)
        self.target_net = CNN().to(device)
        self.target_net.load_state_dict(self.local_net.state_dict())
        self.optimizer = optim.Adam(self.local_net.parameters(), lr=learning_rate)
        self.buffer = ReplayBuffer(buffer_size, batch_size,device)
        self.gamma = gamma
        self.batch_size = batch_size
        self.tau = tau
        self.device = device
        self.trained = False

    def step(self, env, state, action, reward, next_state, done):
        self.buffer.add(state, action, reward, next_state, done)
        if len(self.buffer) >= self.batch_size:
            experiences = self.buffer.sample()
            self.learn(experiences)

    def learn(self, experiences):
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.target_net(next_states).detach().max(1)[0].unsqueeze(1)

        # Compute Q targets for current states
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.local_net(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)

        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update target network
        self.soft_update(self.local_net, self.target_net, self.tau)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

    def act(self, env, state, eps=0.):
        state = torch.from_numpy(state).float().to(self.device).view(1, 14, 8, 8)
        self.local_net.eval()
        with torch.no_grad():
            action_values = self.local_net(state)
        self.local_net.train()

        if random.random() > eps:
            legal_moves = legal_moves_indices(env)
            legal_action_values = action_values[0][legal_moves]
            best_legal_move_index = torch.argmax(legal_action_values).item()
            return legal_moves[best_legal_move_index]
        else:
            return random.choice(legal_moves_indices(env))


### Replay Buffer 정의

In [18]:
class Experience:
    def __init__(self, state, action, reward, next_state, done):
        self.state = state
        self.action = action
        self.reward = reward
        self.next_state = next_state
        self.done = done

    def __getstate__(self):
        return self.state, self.action, self.reward, self.next_state, self.done

    def __setstate__(self, state):
        self.state, self.action, self.reward, self.next_state, self.done = state

class ReplayBuffer:
    def __init__(self, buffer_size, batch_size, device=device):
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.device = device

    def experience(self, state, action, reward, next_state, done):
        return Experience(state, action, reward, next_state, done)
    
    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float().to(self.device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.memory)



## Run Game

2.  save buffer and model 이 true 면 (default) 10000 마다 저장 하는데
    Hyperparameter Optimization 하는 동안은 이 값이 false 가 되어서 저장을 안 하게 됨
    

3.  save_game_log 추가됨.
    이제 오류 없이 매 episode 끝나면 해당 게임의 PGN (메타데이터 포함) 이 저장 됩니다.

### Run Game 함수 정의

In [19]:
class RunGame:
    def __init__(self, dqn_agent1, dqn_agent2, env, num_episodes, episode_movies, eps_start=1.0, eps_end=0.01, eps_decay=0.995, save_model_and_buffer=True):
        self.dqn_agent1 = dqn_agent1
        self.dqn_agent2 = dqn_agent2
        self.env = env
        self.num_episodes = num_episodes
        self.episode_movies = episode_movies
        self.eps_start = eps_start
        self.eps_end = eps_end
        self.eps_decay = eps_decay
        self.save_model_and_buffer = save_model_and_buffer

    @staticmethod

    def save_game_log(env, episode_number, log_dir="data/logs/PGN logs"):
        date_str = datetime.datetime.now().strftime("%Y년 %m월 %d일")
        # Save as PGN
        pgn_filename = f"{date_str}_PGN_LOG.txt"
        pgn_filepath = os.path.join(log_dir, pgn_filename)

        game = chess.pgn.Game()

        # Save the game result
        reward_value = env._reward()
        if reward_value == 1:
            result = "1-0"  # 흰색 승리
        elif reward_value == -1:
            result = "0-1"  # 검은색 승리
        elif reward_value == -0.5:
            result = "1/2-1/2"  # 무승부
        else:
            result = "*"  # 게임 진행 중

        game.headers["Result"] = result
        game.headers["Event"] = "BETAZERO_STUDY"
        game.headers["Site"] = "www.morokei.com/Deeplearning/Chess"
        game.headers["Date"] = f"{date_str}"
        game.headers["Round"] = f"{date_str}_EPISODE_{episode_number}"
        game.headers["White"] = "BETAZERO CARLSEN BOT"
        game.headers["Black"] = "BETAZERO HIKARU BOT"

        # Add moves to the game
        board = env._board
        move_list = board.move_stack
        node = game
        for move in move_list:
            node = node.add_variation(move)

        # Save the game to a file
        with open(pgn_filepath, "a") as pgn_file:
            pgn_file.write(str(game))
            pgn_file.write("\n\n")

    def run(self):
        scores1 = []
        scores2 = []
        scores_window1 = deque(maxlen=100)
        scores_window2 = deque(maxlen=100)
        eps = self.eps_start
        episode_movies = []
        for i_episode in range(1, self.num_episodes + 1):
            state = self.env.reset()
            state = encoder(state).numpy()
            done = False
            score = 0
            moves = []

            if i_episode % 100 == 0:  # Store the episode at every 100th episode
                episode_movie = []

            # Save model and buffer every 10000 episodes
            if self.save_model_and_buffer and i_episode % 1000 == 0:
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                model_name1 = f"BETAZERO_CARLSEN_{timestamp}_EPISODE_{i_episode}.pth"
                model_name2 = f"BETAZERO_HIKARU_{timestamp}_EPISODE_{i_episode}.pth"
                torch.save(self.dqn_agent1.local_net.state_dict(), f'{경로}/data/logs/models/{model_name1}')
                torch.save(self.dqn_agent2.local_net.state_dict(), f'{경로}/data/logs/models/{model_name2}')

                # Save Carlsen's replay buffer
                buffer_name_carlsen = f"{timestamp}_EPISODE_{i_episode}_BETAZERO_CARLSEN_Q_EXPERIENCES.pickle"
                with open(f"{경로}/data/logs/Q_experiences(Buffers)/{buffer_name_carlsen}", "wb") as file:
                    pickle.dump(self.dqn_agent1.buffer, file)

                # Save Hikaru's replay buffer
                buffer_name_hikaru = f"{timestamp}_EPISODE_{i_episode}_BETAZERO_HIKARU_Q_EXPERIENCES.pickle"
                with open(f"{경로}/data/logs/Q_experiences(Buffers)/{buffer_name_hikaru}", "wb") as file:
                    pickle.dump(self.dqn_agent2.buffer, file)

            # Swap agents every episode
            if i_episode % 2 == 0:
                current_agent = self.dqn_agent1
                opponent_agent = self.dqn_agent2
            else:
                current_agent = self.dqn_agent2
                opponent_agent = self.dqn_agent1

            while not done:
                action = current_agent.act(self.env, state, eps)
                next_state, reward, done, info = self.env.step(outcome[action])

                # If the move is illegal, the game does not update the state
                if info['status'] != "Illegal Move":
                    next_state = encoder(next_state).numpy()
                else:
                    reward = -1.0
                    next_state = state

                current_agent.step(self.env, state, action, reward, next_state, done)
                state = next_state
                score += reward

                if i_episode % 100 == 0:  # Store the moves only for the 100th episode
                    moves.append((state, action, next_state, reward, done))

                if done:
                    self.dqn_agent1.trained = True
                    break
            
            # Save and print the move for each episode
            self.save_game_log(self.env, i_episode)
            
            # Swap agents
            current_agent, opponent_agent = opponent_agent, current_agent

            if i_episode % 2 == 0:
                scores_window1.append(score)
                scores1.append(score)
            else:
                scores_window2.append(score)
                scores2.append(score)

            eps = max(self.eps_end, self.eps_decay * eps)

            if i_episode % 100 == 0:
                print(f'\r Episode {i_episode}\tCarlsen Average Score: {np.mean(scores_window1):.2f}, Hikaru Average Score: {np.mean(scores_window2):.2f}')
                print(f"\n Replay buffer size: {len(self.dqn_agent1.buffer)} ({sys.getsizeof(self.dqn_agent1.buffer.memory)/1024/1024:.3f} MB)\n")
                episode_movie.append(moves)

        return scores1, scores2, episode_movies

## OPTUNA

### 하이퍼파라미터 최적화 정의

In [20]:
class HYPERPARAMETER_OPTIMIZING:
    def __init__(self, dqn_agent1, dqn_agent2, env, num_episodes):
        self.dqn_agent1 = dqn_agent1
        self.dqn_agent2 = dqn_agent2
        self.env = env
        self.num_episodes = num_episodes

    def objective(self, trial):
        lr = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
        gamma = trial.suggest_uniform('gamma', 0.9, 0.999)
        tau = trial.suggest_uniform('tau', 0.001, 0.1)
        buffer_size = int(trial.suggest_loguniform('buffer_size', 1e3, 1e7))
        batch_size = 64
        eps_start = 1.0
        eps_end = 0.01
        eps_decay = 0.995

        cnn1 = CNN()
        dqn_agent1 = DQN(cnn1, buffer_size=buffer_size, batch_size=batch_size, gamma=gamma,
                         learning_rate=lr, tau=tau, device=device)

        cnn2 = CNN()
        dqn_agent2 = DQN(cnn2, buffer_size=buffer_size, batch_size=batch_size, gamma=gamma,
                         learning_rate=lr, tau=tau, device=device)

        hyperparameter_optimization_num_episodes = self.num_episodes

        game_runner = RunGame(dqn_agent1, dqn_agent2, env, num_episodes, [])
        scores1, scores2, episode_movies = game_runner.run()
        return np.mean(scores1)


    def optimize(self, n_trials):
        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=n_trials)

        print('Number of finished trials:', len(study.trials))
        print('Best trial:', study.best_trial.params)


## Model Save & Load 함수 정의

### Save Model 함수 정의

In [21]:
def save_model(dqn_agent):
    if not dqn_agent.trained:
        print("저장할 모델이 없습니다.")
        return

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    model_filename = f"BETAZERO_{timestamp}_SAVED_EPISODE.pth"
    torch.save(dqn_agent.local_net.state_dict(), f'{경로}/data/logs/models/{model_filename}')


### Load Model 함수 정의

In [22]:
class LoadModel:
    def __init__(self, path):
        # Find the most recent model and buffer files
        model_files = os.listdir(f"{path}/data/logs/models/")
        buffer_files = os.listdir(f"{path}/data/logs/Q_experiences(Buffers)/")

        carlsen_model_files = [file for file in model_files if "BETAZERO_CARLSEN" in file]
        hikaru_model_files = [file for file in model_files if "BETAZERO_HIKARU" in file]

        carlsen_buffer_files = [file for file in buffer_files if "BETAZERO_CARLSEN" in file]
        hikaru_buffer_files = [file for file in buffer_files if "BETAZERO_HIKARU" in file]

        # Extract episode numbers from the file names
        carlsen_model_episodes = [int(re.search("EPISODE_(\d+)", file).group(1)) for file in carlsen_model_files]
        hikaru_model_episodes = [int(re.search("EPISODE_(\d+)", file).group(1)) for file in hikaru_model_files]

        carlsen_buffer_episodes = [int(re.search("EPISODE_(\d+)", file).group(1)) for file in carlsen_buffer_files]
        hikaru_buffer_episodes = [int(re.search("EPISODE_(\d+)", file).group(1)) for file in hikaru_buffer_files]

        # Find the most recent episodes
        carlsen_most_recent_episode = max(carlsen_model_episodes)
        hikaru_most_recent_episode = max(hikaru_model_episodes)

        carlsen_model_file = carlsen_model_files[carlsen_model_episodes.index(carlsen_most_recent_episode)]
        hikaru_model_file = hikaru_model_files[hikaru_model_episodes.index(hikaru_most_recent_episode)]

        carlsen_buffer_file = carlsen_buffer_files[carlsen_buffer_episodes.index(carlsen_most_recent_episode)]
        hikaru_buffer_file = hikaru_buffer_files[hikaru_buffer_episodes.index(hikaru_most_recent_episode)]

        # Load the most recent models and buffers
        self._load_model_and_buffer(carlsen_model_file, carlsen_buffer_file, 1)
        self._load_model_and_buffer(hikaru_model_file, hikaru_buffer_file, 2)

    def _load_model_and_buffer(self, model_filename, buffer_filename, model_number):
        # Load the model
        model = CNN()
        model.load_state_dict(torch.load(f'{경로}/data/logs/models/{model_filename}'))
        model.to(device)  # GPU에서 실행할 경우 필요
        model.eval()  # 추론 모드로 전환

        # Load the replay buffer
        with open(f"{경로}/data/logs/Q_experiences(Buffers)/{buffer_filename}", "rb") as file:
            buffer = pickle.load(file)

        if model_number == 1:
            self.model1 = model
            self.buffer1 = buffer
        else:
            self.model2 = model
            self.buffer2 = buffer
    def infer_q_values(self, state, model_number):
        state_tensor = torch.from_numpy(state).float().to(device).view(1, 14, 8, 8)
        if model_number == 1:
            q_values = self.model1(state_tensor)
        else:
            q_values = self.model2(state_tensor)
        return q_values

    def select_best_action(self, state, model_number):
        q_values = self.infer_q_values(state, model_number)
        best_action = np.argmax(q_values.cpu().data.numpy())
        return best_action

    def evaluate(self, env, num_episodes):
        total_rewards = []

        for episode in range(num_episodes):
            state = env.reset()
            state = encoder(state).numpy()
            done = False
            episode_reward = 0

            while not done:
                best_action = self.select_best_action(state)
                next_state, reward, done, info = env.step(outcome[best_action])

                if info['status'] != "Illegal Move":
                    next_state = encoder(next_state).numpy()

                episode_reward += reward
                state = next_state

                if done:
                    break

            total_rewards.append(episode_reward)

        return total_rewards

    def interactive_menu(self, env):
        while True:
            print("\n1: Q 값 추론")
            print("2: 최적의 행동 선택")
            print("3: 평가하기")
            print("4: 추가 학습")
            print("0: 종료")
            option = input("Enter the option number (1, 2, 3, 4, or 0): ")
            
            if option == "1":
                model_number = int(input("모델 번호를 선택하세요 (1: CARLSEN, 2: HIKARU): "))
                sample_state = env.reset()
                encoded_state = encoder(sample_state).numpy()
                q_values = self.infer_q_values(encoded_state, model_number)
                print("Q values:", q_values)
            elif option == "2":
                model_number = int(input("모델 번호를 선택하세요 (1: CARLSEN, 2: HIKARU): "))
                sample_state = env.reset()
                encoded_state = encoder(sample_state).numpy()
                best_action = self.select_best_action(encoded_state, model_number)
                print("Best action:", best_action)
            elif option == "3":
                num_evaluation_episodes = 100
                evaluation_rewards = self.evaluate(env, num_evaluation_episodes)
                average_reward = np.mean(evaluation_rewards)
                print(f"모델의 평균 보상: {average_reward}")
            elif option == "4":
                cnn1 = CNN()
                cnn2 = CNN()
                dqn_agent1 = DQN(cnn1, device=device)
                dqn_agent2 = DQN(cnn2, device=device)
                num_episodes = 1000000
                
                game_runner = RunGame(dqn_agent1, dqn_agent2, env, num_episodes, [])
                scores1, scores2, episode_movies = game_runner.run()
            
                
            elif option == "0":
                break
            else:
                print("Invalid input. Please enter 1, 2, 3, 4, or 0.")


## MOVIE 정의

In [23]:
import pygame

def betazero_vs_betazero_movie(episodes, index):
    pygame.init()
    pygame.mixer.init()
    moves = episodes[index]

    # Display settings
    WIDTH, HEIGHT = 800, 800
    BOARD_SIZE = 8
    SQ_SIZE = WIDTH // BOARD_SIZE

    # Colors
    WHITE = (216, 182, 142)
    BLACK = (101, 67, 33)
    GREY = (128, 128, 128)

    # Initialize mixer and load music
    music_files = [
        ""+경로+"/data/UI/bgm/Jazz Music #1.mp3",
        ""+경로+"/data/UI/bgm/Jazz Music #2.mp3",
        ""+경로+"/data/UI/bgm/Jazz Music #3.mp3",
        ""+경로+"/data/UI/bgm/Jazz Music #4.mp3",
        ""+경로+"/data/UI/bgm/Jazz Music #5.mp3",
    ]
    MUSIC_ENDEVENT = pygame.USEREVENT + 1
    pygame.mixer.music.set_endevent(MUSIC_ENDEVENT)
    current_music_index = random.randint(0, 2)
    pygame.mixer.music.load(music_files[current_music_index])
    pygame.mixer.music.set_volume(0.2)
    pygame.mixer.music.play()

    def draw_board(screen, board):
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                color = WHITE if (r + c) % 2 == 0 else BLACK
                pygame.draw.rect(screen, color, (c * SQ_SIZE, r * SQ_SIZE, SQ_SIZE, SQ_SIZE))
                piece = board.piece_at((BOARD_SIZE - 1 - r) * BOARD_SIZE + c)
                if piece:
                    screen.blit(PIECES[str(piece)], (c * SQ_SIZE, r * SQ_SIZE))

    def main(episodes, index):
        screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Betazero VS Betazero Movie")

        global PIECES
        PIECES = {}
        piece_path = ""+경로+"/data/UI/기물 이미지"
        for piece_type in chess.PIECE_TYPES:
            for color in [chess.WHITE, chess.BLACK]:
                piece = chess.Piece(piece_type, color)
                name = f"Chess_{piece.symbol()}{'w' if color else 'd'}.png"
                filepath = os.path.join(piece_path, name)
                image = pygame.image.load(filepath).convert_alpha()
                image = pygame.transform.scale(image, (SQ_SIZE, SQ_SIZE))
                PIECES[str(piece)] = image

        moves = episodes[index]
        board = chess.Board()
        for move in moves:
            board.push(move)
            draw_board(screen, board)
            pygame.display.update()
            pygame.time.delay(500)

        pygame.time.delay(2000)
        pygame.quit()

    main(episodes, index)

pygame 2.1.3.dev8 (SDL 2.0.22, Python 3.10.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 실행 !

In [24]:
def 실행():
    while True:
        print("\n1: 모델 만들기")
        print("2: 하이퍼 파라미터 최적화")
        print("3: 모델 저장하기")
        print("4: 모델 불러오기")
        print("0: 종료")

        choice = input("Enter the number (1, 2, 3, 4, or 0): ")
        
        cnn1 = CNN()
        cnn2 = CNN()
        dqn_agent1 = DQN(cnn1, device=device)
        dqn_agent2 = DQN(cnn2, device=device)

        if choice == "1":
            print("\n모델 학습을 시작합니다.")
        elif choice == "2":
            print("\n하이퍼 파라미터 최적화를 시작합니다.")
        elif choice == "3":
            print("\n모델을 저장합니다.")
        elif choice == "4":
            print("\n모델을 불러옵니다.")
        elif choice == "0":
            break
        else:
            print("Invalid input. Please enter 1, 2, 3, 4, or 0.")
            continue

        for i in range(3, 0, -1):
            sys.stdout.write(f"\r{i}...")
            sys.stdout.flush()
            time.sleep(1)

        sys.stdout.write("\r")
        sys.stdout.flush()
        print("\n")

        if choice == "1":
            episode_movies = []
            num_episodes = 100000
            
            game_runner = RunGame(dqn_agent1, dqn_agent2, env, num_episodes, [])
            scores1, scores2, episode_movies = game_runner.run()
            # Play the movies
            for idx, episode in enumerate(episode_movies):
                betazero_vs_betazero_movie(episode, idx)

        elif choice == "2":
            num_episodes = 20000
            opt = HYPERPARAMETER_OPTIMIZING(dqn_agent1, dqn_agent2, env, num_episodes)
            
            opt.optimize(n_trials=10)
        elif choice == "3":
            save_model(dqn_agent1)
        elif choice == "4":
            model_filename = input("Enter the model filename (without .pth extension): ")
            loaded_model = LoadModel(model_filename)
            loaded_model.interactive_menu(env)

if __name__ == "__main__":
    실행()



1: 모델 만들기
2: 하이퍼 파라미터 최적화
3: 모델 저장하기
4: 모델 불러오기
0: 종료
Enter the number (1, 2, 3, 4, or 0): 1

모델 학습을 시작합니다.
1...

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\ojs53\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ojs53\AppData\Local\Temp\ipykernel_14628\400420255.py", line 62, in <module>
    실행()
  File "C:\Users\ojs53\AppData\Local\Temp\ipykernel_14628\400420255.py", line 44, in 실행
    scores1, scores2, episode_movies = game_runner.run()
  File "C:\Users\ojs53\AppData\Local\Temp\ipykernel_14628\1433714735.py", line 108, in run
    current_agent.step(self.env, state, action, reward, next_state, done)
  File "C:\Users\ojs53\AppData\Local\Temp\ipykernel_14628\989182782.py", line 18, in step
    self.learn(experiences)
  File "C:\Users\ojs53\AppData\Local\Temp\ipykernel_14628\989182782.py", line 24, in learn
    Q_targets_next = self.target_net(next_states).detach().max(1)[0].unsqueeze(1)
  File "C:\Users\ojs53\AppData\Local\Programs\Python\Python310\lib\s

In [ ]:
env._reward()

## EVALUATION

### HUMAN VS BETAZERO

In [ ]:
def 베타제로랑한판():
    import pygame
    import chess
    import chess.engine
    import chess.svg
    import os
    import sys
    import io
    from reportlab.graphics import renderPM
    from svglib.svglib import svg2rlg
    
    def load_latest_betazero_model(model_directory):
        model_files = glob.glob(os.path.join(model_directory, "BETAZERO_CARLSEN_*.pth"))
        latest_model_file = max(model_files, key=os.path.getctime)
        print(latest_model_file)
        return latest_model_file

    # Load BETAZERO model
    model_directory = ""+경로+"/data/logs/models"
    model_path = load_latest_betazero_model(model_directory)

    # Load BETAZERO model
    cnn = CNN()
    betazero_agent = DQN(cnn)
    betazero_agent.local_net.load_state_dict(torch.load(model_path))
    betazero_agent.local_net.eval()


    def betazero_action(state, agent):
        state_tensor = encoder(state).unsqueeze(0).squeeze(1).to(agent.device)
        action_values = agent.local_net(state_tensor)
        _, action = torch.max(action_values, dim=1)
        return action.item()


    pygame.init()

    # Display settings
    WIDTH, HEIGHT = 800, 800
    BOARD_SIZE = 8
    SQ_SIZE = WIDTH // BOARD_SIZE

    # Colors
    WHITE = (216, 182, 142)
    BLACK = (101, 67, 33)
    GREY = (128, 128, 128)

    # 탭 화면
    TAB_WIDTH = 400
    TAB_HEIGHT = HEIGHT
    TAB_X = WIDTH
    TAB_Y = 0

    def svg_to_surface(svg_data, size):
        svg_io = io.StringIO(svg_data)
        svg_image = svg2rlg(svg_io)
        buffer = io.BytesIO()
        renderPM.drawToFile(svg_image, buffer, fmt="PNG")
        buffer.seek(0)
        return pygame.image.load(io.BytesIO(buffer.read()))

    def draw_board(screen, board, moves, selected, player_color, row, col):
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                color = WHITE if (r + c) % 2 == 0 else BLACK
                pygame.draw.rect(screen, color, (c * SQ_SIZE, r * SQ_SIZE, SQ_SIZE, SQ_SIZE))
                piece = board.piece_at((BOARD_SIZE - 1 - r) * BOARD_SIZE + c)
                if piece:
                    screen.blit(PIECES[str(piece)], (c * SQ_SIZE, r * SQ_SIZE))
        if selected is not None:
            r, c = divmod(selected, BOARD_SIZE)
            pygame.draw.rect(screen, (255, 0, 0), (c * SQ_SIZE, (BOARD_SIZE - 1 - r) * SQ_SIZE, SQ_SIZE, SQ_SIZE), 5)
            for move in moves:
                r, c = divmod(move.to_square, BOARD_SIZE)
                pygame.draw.circle(screen, GREY, (c * SQ_SIZE + SQ_SIZE // 2, (BOARD_SIZE - 1 - r) * SQ_SIZE + SQ_SIZE // 2),
                   SQ_SIZE // 4)
    def main():
        screen = pygame.display.set_mode((WIDTH + TAB_WIDTH, HEIGHT))
        tab_surface = pygame.Surface((TAB_WIDTH, TAB_HEIGHT))
        tab_surface.fill((0, 0, 0))

        #pygame.mixer.music.load(''+경로+'/data/UI/bgm/what am i to say _ sum 41.mp3')
        #pygame.mixer.music.set_volume(0.2)
        #pygame.mixer.music.play(-1)
        FONT = pygame.font.SysFont("arial", 32)
        pygame.display.set_caption("모로케이의 얼렁뚱땅 체스한판")

        global PIECES
        PIECES = {}
        piece_path = ""+경로+"/data/UI/기물 이미지"
        for piece_type in chess.PIECE_TYPES:
            for color in [chess.WHITE, chess.BLACK]:
                piece = chess.Piece(piece_type, color)
                name = f"Chess_{piece.symbol()}{'w' if color else 'd'}.png"
                filepath = os.path.join(piece_path, name)
                image = pygame.image.load(filepath).convert_alpha()
                image = pygame.transform.scale(image, (SQ_SIZE, SQ_SIZE))
                PIECES[str(piece)] = image

        board = chess.Board()
        player_color = chess.WHITE

        clock = pygame.time.Clock()

        selected = None
        moves = []  # possible moves for the selected piece
        game_running = True
        game_over = False
        row, col = None, None
        while game_running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    game_running = False
                elif not game_over:
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        # get the square clicked by the user
                        x, y = pygame.mouse.get_pos()
                        col = x // SQ_SIZE
                        row = y // SQ_SIZE
                        # select a piece
                        piece = board.piece_at((BOARD_SIZE - 1 - row) * BOARD_SIZE + col)
                        if piece:
                            if selected == None and piece.color == player_color:
                                selected = (BOARD_SIZE - 1 - row) * BOARD_SIZE + col
                                from_mask = chess.BB_SQUARES[selected]
                                moves = list(board.generate_legal_moves(from_mask=from_mask))
                    elif event.type == pygame.MOUSEBUTTONUP:
                        if selected != None:
                            # get the ending square
                            x, y = pygame.mouse.get_pos()
                            col = x // SQ_SIZE
                            row = y // SQ_SIZE
                            dest = (BOARD_SIZE - 1 - row) * BOARD_SIZE + col

                            # attempt to make the move
                            move = chess.Move(selected, dest)
                            if move in moves:
                                board.push(move)
                                print(board)
                                # get the betazero's move
                                betazero_move = betazero_action(board, betazero_agent)
                                print("Betazero move in main:", betazero_move)
                                move = index_to_move(betazero_move)
                                print("Converted move:", move)
                                print(board.is_game_over())
                                if move in board.legal_moves:
                                    board.push(move)
                                else:
                                    print(f"{move} is not a legal move.")

                            # reset the selected piece
                            selected = None
                            moves = []

            draw_board(screen, board, moves, selected, player_color, col, row)
            screen.blit(tab_surface, (TAB_X, TAB_Y))

            # check for endgame
            if board.is_game_over():
                game_over = True  # Set game_over flag to True
                result = board.result()
                if result == "0-1":
                    text = FONT.render("You lose", True, (255, 0, 0))
                elif result == "1-0":
                    text = FONT.render("You win", True, (0, 255, 0))
                else:
                    text = FONT.render("Draw", True, (128, 128, 128))

                tab_surface.blit(text, (TAB_WIDTH // 2 - text.get_width() // 2, TAB_HEIGHT // 2 - text.get_height() // 2))
                pygame.display.update()

            pygame.display.update()
            clock.tick(60)
    main()


### STOCKFISH VS BETAZERO

In [ ]:
# Play against Stockfish
def betazero_vs_stockfish():
    import chess.engine

    engine = chess.engine.SimpleEngine.popen_uci(""+경로+"/data/UI/stockfish/stockfish-windows-2022-x86-64-avx2.exe")

    done = False
    observation = env.reset()
    while not done:
        # Get action from the agent
        action = dqn_agent.act(observation)

        # Play the action
        fen = env.observation_to_fen(observation)
        board = chess.Board(fen)
        engine_move = engine.play(board, chess.engine.Limit(time=2.0))
        move = gym_chess.utils.uci_to_action(str(engine_move.move), board.turn)
        observation, reward, done, info = env.step(move)

        # Render the board
        env.render()

        if done:
            # Print the game result
            print("Game over: ", info["result"])

    engine.quit()


### STOCKFISH VS HUMAN

In [ ]:
def 스탁피시랑한판():
    import pygame
    import chess
    import chess.engine
    import chess.svg
    import os
    import sys
    import io
    from reportlab.graphics import renderPM
    from svglib.svglib import svg2rlg

    pygame.init()

    # Display settings
    WIDTH, HEIGHT = 800, 800
    BOARD_SIZE = 8
    SQ_SIZE = WIDTH // BOARD_SIZE

    # Colors
    WHITE = (216, 182, 142)
    BLACK = (101, 67, 33)
    GREY = (128, 128, 128)

    # 탭 화면
    TAB_WIDTH = 400
    TAB_HEIGHT = HEIGHT
    TAB_X = WIDTH
    TAB_Y = 0

    def svg_to_surface(svg_data, size):
        svg_io = io.StringIO(svg_data)
        svg_image = svg2rlg(svg_io)
        buffer = io.BytesIO()
        renderPM.drawToFile(svg_image, buffer, fmt="PNG")
        buffer.seek(0)
        return pygame.image.load(io.BytesIO(buffer.read()))

    def draw_board(screen, board, moves, selected, player_color, row, col):
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                color = WHITE if (r + c) % 2 == 0 else BLACK
                pygame.draw.rect(screen, color, (c * SQ_SIZE, r * SQ_SIZE, SQ_SIZE, SQ_SIZE))
                piece = board.piece_at((BOARD_SIZE - 1 - r) * BOARD_SIZE + c)
                if piece:
                    screen.blit(PIECES[str(piece)], (c * SQ_SIZE, r * SQ_SIZE))
        if selected is not None:
            r, c = divmod(selected, BOARD_SIZE)
            pygame.draw.rect(screen, (255, 0, 0), (c * SQ_SIZE, (BOARD_SIZE - 1 - r) * SQ_SIZE, SQ_SIZE, SQ_SIZE), 5)
            for move in moves:
                r, c = divmod(move.to_square, BOARD_SIZE)
                pygame.draw.circle(screen, GREY, (c * SQ_SIZE + SQ_SIZE // 2, (BOARD_SIZE - 1 - r) * SQ_SIZE + SQ_SIZE // 2),
                                   SQ_SIZE // 4)

    def main():
        screen = pygame.display.set_mode((WIDTH + TAB_WIDTH, HEIGHT))
        tab_surface = pygame.Surface((TAB_WIDTH, TAB_HEIGHT))
        tab_surface.fill((0, 0, 0))

        pygame.mixer.music.load(""+경로+"/data/UI/bgm/what am i to say _ sum 41.mp3")
        pygame.mixer.music.set_volume(0.2)
        pygame.mixer.music.play(-1)
        FONT = pygame.font.SysFont("arial", 32)
        pygame.display.set_caption("모로케이의 얼렁뚱땅 체스한판")

        global PIECES
        PIECES = {}
        piece_path = ""+경로+"/data/UI/기물 이미지"
        for piece_type in chess.PIECE_TYPES:
            for color in [chess.WHITE, chess.BLACK]:
                piece = chess.Piece(piece_type, color)
                name = f"Chess_{piece.symbol()}{'w' if color else 'd'}.png"
                filepath = os.path.join(piece_path, name)
                image = pygame.image.load(filepath).convert_alpha()
                image = pygame.transform.scale(image, (SQ_SIZE, SQ_SIZE))
                PIECES[str(piece)] = image

        board = chess.Board()
        engine = chess.engine.SimpleEngine.popen_uci(""+경로+"/data/UI/stockfish/stockfish-windows-2022-x86-64-avx2.exe")
        player_color = chess.WHITE

        clock = pygame.time.Clock()

        selected = None
        moves = []  # possible moves for the selected piece
        game_running = True
        game_over = False
        row, col = None, None
        while game_running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    game_running = False
                elif not game_over:
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        # get the square clicked by the user
                        x,y = pygame.mouse.get_pos()
                        col = x // SQ_SIZE
                        row = y // SQ_SIZE
                        # select a piece
                        piece = board.piece_at((BOARD_SIZE - 1 - row) * BOARD_SIZE + col)
                        if piece:
                            if selected == None and piece.color == player_color:
                                selected = (BOARD_SIZE - 1 - row) * BOARD_SIZE + col
                                from_mask = chess.BB_SQUARES[selected]
                                moves = list(board.generate_legal_moves(from_mask=from_mask))
                    elif event.type == pygame.MOUSEBUTTONUP:
                        if selected != None:
                            # get the ending square
                            x,y = pygame.mouse.get_pos()
                            col = x // SQ_SIZE
                            row = y // SQ_SIZE
                            dest = (BOARD_SIZE - 1 - row) * BOARD_SIZE + col

                            # attempt to make the move
                            move = chess.Move(selected, dest)
                            if move in moves:
                                board.push(move)

                                # get the engine's move
                                result = engine.play(board, chess.engine.Limit(time=0.1))
                                board.push(result.move)

                            # reset the selected piece
                            selected = None
                            moves = []

            draw_board(screen, board, moves, selected, player_color, col, row)
            screen.blit(tab_surface, (TAB_X, TAB_Y))

            # check for endgame
            if board.is_game_over():
                game_over = True  # Set game_over flag to True
                result = board.result()
                if result == "0-1":
                    text = FONT.render("You lose", True, (255, 0, 0))
                elif result == "1-0":
                    text = FONT.render("You win", True, (0, 255, 0))
                else:
                    text = FONT.render("Draw", True, (128, 128, 128))

                tab_surface.blit(text, (TAB_WIDTH // 2 - text.get_width() // 2, TAB_HEIGHT // 2 - text.get_height() // 2))
                pygame.display.update()

            pygame.display.update()
            clock.tick(60)
    main()


### BETAZERO VS BETAZERO

### STOCKFISH VS STOCKFISH

In [ ]:
def stockfish_vs_stockfish():
    import pygame ; import chess  ; import chess.engine  ; import os  ;import sys
    import io ; import random ; import datetime ; import time
    인풋받기 = int(input("몇게임돌릴래? \n"))
    time.sleep(3)
    pygame.init()
    
    # Display settings
    WIDTH, HEIGHT = 800, 800
    BOARD_SIZE = 8
    SQ_SIZE = WIDTH // BOARD_SIZE
    WHITE = (216, 182, 142)
    BLACK = (101, 67, 33)
    GREY = (128, 128, 128)
    TAB_WIDTH = 400
    TAB_HEIGHT = HEIGHT
    TAB_X = WIDTH
    TAB_Y = 0

    pygame.display.set_caption("Stockfish vs Stockfish")
    screen = pygame.display.set_mode((WIDTH + TAB_WIDTH, HEIGHT))

    # Initialize mixer and load music
    pygame.mixer.init()
    music_files = [
        ""+경로+"/data/UI/bgm/2pac california love.mp3",
        ""+경로+"/data/UI/bgm/all eyes on me _ 2pac.mp3",
        ""+경로+"/data/UI/bgm/dear mama - 2pac.mp3"
    ]
    MUSIC_ENDEVENT = pygame.USEREVENT + 1
    pygame.mixer.music.set_endevent(MUSIC_ENDEVENT)
    current_music_index = random.randint(0, 2)
    pygame.mixer.music.load(music_files[current_music_index])
    pygame.mixer.music.set_volume(0.2)
    pygame.mixer.music.play()

    FONT = pygame.font.SysFont("arial", 32)
    tab_surface = pygame.Surface((TAB_WIDTH, TAB_HEIGHT))
    tab_surface.fill((0, 0, 0))

    global PIECES
    PIECES = {}
    piece_path = ""+경로+"/data/UI/기물 이미지"
    for piece_type in chess.PIECE_TYPES:
        for color in [chess.WHITE, chess.BLACK]:
            piece = chess.Piece(piece_type, color)
            name = f"Chess_{piece.symbol()}{'w' if color else 'd'}.png"
            filepath = os.path.join(piece_path, name)
            image = pygame.image.load(filepath).convert_alpha()
            image = pygame.transform.scale(image, (SQ_SIZE, SQ_SIZE))
            PIECES[str(piece)] = image

    board = chess.Board()
    engine = chess.engine.SimpleEngine.popen_uci(""+경로+"/data/UI/stockfish/stockfish-windows-2022-x86-64-avx2.exe")

    clock = pygame.time.Clock()

    total_games = 인풋받기
    white_wins = 0
    black_wins = 0
    draws = 0
    game_over = False
    
    for game in range(total_games):
        board = chess.Board()
        game_running = True
        game_over = False
        pygame.time.delay(3000)
        while game_running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    game_running = False
                elif event.type == MUSIC_ENDEVENT:
                    current_music_index = (current_music_index + 1) % len(music_files)
                    pygame.mixer.music.load(music_files[current_music_index])
                    pygame.mixer.music.play()

            if not game_over:
                result = engine.play(board, chess.engine.Limit(time=0.1))
                if result.move is not None:
                    board.push(result.move)

                for r in range(BOARD_SIZE):
                    for c in range(BOARD_SIZE):
                        color = WHITE if (r + c) % 2 == 0 else BLACK
                        pygame.draw.rect(screen, color, (c * SQ_SIZE, r * SQ_SIZE, SQ_SIZE, SQ_SIZE))
                        piece = board.piece_at((BOARD_SIZE - 1 - r) * BOARD_SIZE + c)
                        if piece:
                            screen.blit(PIECES[str(piece)], (c * SQ_SIZE, r * SQ_SIZE))

                pygame.display.update()

                if board.is_game_over():
                    game_over = True
                    game_running = False
                    result = board.result()
                    if result == "0-1":
                        text = FONT.render("Black wins", True, (255, 0, 0))
                        black_wins += 1
                    elif result == "1-0":
                        text = FONT.render("White wins", True, (0, 255, 0))
                        white_wins += 1
                    else:
                        text = FONT.render("Draw", True, (128, 128, 128))
                        draws += 1

                    tab_surface.blit(text, (TAB_WIDTH // 2 - text.get_width() // 2, TAB_HEIGHT // 2 - text.get_height() // 2))
                    screen.blit(tab_surface, (WIDTH, 0))
                    pygame.display.update()
                    pygame.time.delay(3000)
                    tab_surface.fill((0, 0, 0), (0, TAB_HEIGHT // 2 - text.get_height() // 2, TAB_WIDTH, text.get_height()))

            else:
                game_over = False

            scoreboard_title = FONT.render("SCOREBOARD:", True, (255, 255, 255))
            scoreboard = FONT.render(f"WHITE {white_wins} - {black_wins} BLACK", True, (255, 255, 255))
            scoreboard_draw = FONT.render(f"DRAW: {draws}", True, (255, 255, 255))
            tab_surface.fill((0, 0, 0), (0, 0, TAB_WIDTH, 150))
            tab_surface.blit(scoreboard_title, (TAB_WIDTH // 2 - scoreboard_title.get_width() // 2, 10))
            tab_surface.blit(scoreboard, (TAB_WIDTH // 2 - scoreboard.get_width() // 2, 70))
            tab_surface.blit(scoreboard_draw, (TAB_WIDTH // 2 - scoreboard_draw.get_width() // 2, 110))
            FONT_SMALL = pygame.font.SysFont("arial", 20)
            FONT_MED = pygame.font.SysFont("arial", 24)
            # Display current time and additional information
            current_time = datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S %Z%z")
            time_text = FONT_SMALL.render(f" Current time: {current_time} KST (UTC+9) ", True, GREY)
            morokei_text = FONT_MED.render(" Morokei's STOCKFISH GRINDER V 7.0 ", True, GREY)
            sf_vs_sf_text = FONT_MED.render(" STOCKFISH 15.0 VS STOCKFISH 15.0 ", True, GREY)

            tab_surface.fill((0, 0, 0), (0, TAB_HEIGHT - 130, TAB_WIDTH, 130))
            tab_surface.blit(time_text, (TAB_WIDTH // 2 - time_text.get_width() // 2, TAB_HEIGHT - 120))
            tab_surface.blit(morokei_text, (TAB_WIDTH // 2 - morokei_text.get_width() // 2, TAB_HEIGHT - 80))
            tab_surface.blit(sf_vs_sf_text, (TAB_WIDTH // 2 - sf_vs_sf_text.get_width() // 2, TAB_HEIGHT - 40))
            screen.blit(tab_surface, (TAB_X, TAB_Y))
            pygame.display.update()
            clock.tick(60)

    engine.quit()

# 게임 실행 !

In [ ]:
def 게임_메뉴():
    while True:
        print("\n1: 베타제로와 한 판")
        print("2: 베타제로 vs 스톡피시")
        print("3: 스톡피시와 한 판")
        print("4: 베타제로 vs 베타제로")
        print("5: 스톡피시 vs 스톡피시")
        print("0: 종료")

        choice = input("Enter the number (1, 2, 3, 4, 5, or 0): ")

        if choice in ["1", "2", "3", "4", "5 \n"]:
            for i in range(3, 0, -1):
                sys.stdout.write(f"\r{i}...")
                sys.stdout.flush()
                time.sleep(1)

            sys.stdout.write("\r")
            sys.stdout.flush()
            print("\n")

            if choice == "1":
                베타제로랑한판()
            elif choice == "2":
                betazero_vs_stockfish()
            elif choice == "3":
                스탁피시랑한판()
            elif choice == "4":
                betazero_vs_betazero()
            elif choice == "5":
                stockfish_vs_stockfish()
        elif choice == "0":
            print("종료합니다.")
            break
        else:
            print("Invalid input. Please enter 1, 2, 3, 4, 5, or 0.")

if __name__ == "__main__":
    main_menu()
